<a href="https://colab.research.google.com/github/j2nhyeok/self-study/blob/main/type2_3rd(blank).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형2
- 빅데이터 분석기사 3회 실기
- 제공된 데이터
    - train.csv
    - test.csv
- 예측할 컬럼 : TravelInsurance

### 시험에서 제공된 코드
~~~
import pandas as pd
a = pd.read_csv("train.csv")
b = pd.read_csv('test.csv')

pd.DataFrame(변수).to_csv('000000000.csv')
~~~


### 보험가입 확률을 묻는 문제
~~~
# csv 형태
# index,y_pred
# 0,0.152467
# 1,0.961511
# 2,0.151313
# 3,0.236632
...
# 494,0.121372
# 495,0.965155
# 496,0.986405

~~~


## 데이터 불러오기

In [16]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## EDA

In [17]:
# 크기 확인
print(train.shape, test.shape)

(1490, 10) (497, 9)


In [18]:
# 샘플 확인
train.head()

,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,888,28,Private Sector/Self Employed,Yes,1250000,6,1,No,No,0
1,1308,31,Private Sector/Self Employed,Yes,1250000,7,1,No,No,0
2,151,29,Private Sector/Self Employed,Yes,1200000,7,0,No,No,1
3,396,33,Government Sector,Yes,650000,6,1,No,No,1
4,135,28,Private Sector/Self Employed,Yes,800000,6,0,No,Yes,1


In [19]:
# 타입 확인
train.nunique()

Unnamed: 0             1490
Age                      11
Employment Type           2
GraduateOrNot             2
AnnualIncome             30
FamilyMembers             8
ChronicDiseases           2
FrequentFlyer             2
EverTravelledAbroad       2
TravelInsurance           2
dtype: int64

In [20]:
# 결측치 확인 train
print(train.isnull().sum())

Unnamed: 0             0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
TravelInsurance        0
dtype: int64


In [21]:
# 결측치 확인 test
print(test.isnull().sum())

Unnamed: 0             0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64


In [22]:
# 기초 통계 획인
print(train.describe())

        Unnamed: 0          Age  AnnualIncome  FamilyMembers  ChronicDiseases  \
count  1490.000000  1490.000000  1.490000e+03    1490.000000      1490.000000   
mean    992.277181    29.600000  9.310738e+05       4.755705         0.280537   
std     566.637006     2.887829  3.763057e+05       1.603613         0.449412   
min       0.000000    25.000000  3.000000e+05       2.000000         0.000000   
25%     502.250000    28.000000  6.000000e+05       4.000000         0.000000   
50%     994.500000    29.000000  9.000000e+05       5.000000         0.000000   
75%    1479.750000    32.000000  1.250000e+06       6.000000         1.000000   
max    1983.000000    35.000000  1.800000e+06       9.000000         1.000000   

       TravelInsurance  
count      1490.000000  
mean          0.352349  
std           0.477862  
min           0.000000  
25%           0.000000  
50%           0.000000  
75%           1.000000  
max           1.000000  


In [23]:
# 기초 통계 획인 object
print(train.describe(include='O'))

                     Employment Type GraduateOrNot FrequentFlyer  \
count                           1490          1490          1490   
unique                             2             2             2   
top     Private Sector/Self Employed           Yes            No   
freq                            1056          1270          1175   

       EverTravelledAbroad  
count                 1490  
unique                   2  
top                     No  
freq                  1209  


In [24]:
# 타겟(레이블)
target = train.pop('TravelInsurance')
target.head()

0    0
1    0
2    1
3    1
4    1
Name: TravelInsurance, dtype: int64


## 데이터 전처리 및 피처엔지니어링

In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
dtypes: int64(5), object(4)
memory usage: 104.9+ KB


In [26]:
# 수치형 데이터와 범주형 데이터 분리
train_numeric = train.select_dtypes(exclude = 'object')
train_categoric = train.select_dtypes(include = 'object')
test_numeric = test.select_dtypes(exclude = 'object')
test_categoric = test.select_dtypes(include = 'object')

print(train_numeric.shape, train_categoric.shape, test_numeric.shape, test_numeric.shape)
print(train_numeric.head())
print(test_numeric.head())

(1490, 5) (1490, 4) (497, 5) (497, 5)
   Unnamed: 0  Age  AnnualIncome  FamilyMembers  ChronicDiseases
0         888   28       1250000              6                1
1        1308   31       1250000              7                1
2         151   29       1200000              7                0
3         396   33        650000              6                1
4         135   28        800000              6                0
   Unnamed: 0  Age  AnnualIncome  FamilyMembers  ChronicDiseases
0         920   32        650000              5                0
1         777   27        500000              5                0
2         696   32        350000              2                0
3         907   26       1400000              6                0
4         675   27        350000              3                0


In [27]:
# 수치형 변수 스케일링
from sklearn.preprocessing import StandardScaler
cols = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']
scale = StandardScaler()
train_numeric[cols] = scale.fit_transform(train_numeric[cols])
test_numeric[cols] = scale.transform(test_numeric[cols])

In [28]:
# 범주형 변수 인코딩

train_categoric = pd.get_dummies(train_categoric)
test_categoric = pd.get_dummies(test_categoric)


In [29]:
# 분리한 데이터 다시 합침
train = pd.concat([train_numeric, train_categoric], axis = 1)
test = pd.concat([test_numeric, test_categoric], axis = 1)
print(train.head())

   Unnamed: 0       Age  AnnualIncome  FamilyMembers  ChronicDiseases  \
0         888 -0.554235      0.847803       0.776193         1.601435   
1        1308  0.484956      0.847803       1.399994         1.601435   
2         151 -0.207838      0.714888       1.399994        -0.624440   
3         396  1.177750     -0.747180       0.776193         1.601435   
4         135 -0.554235     -0.348434       0.776193        -0.624440   

   Employment Type_Government Sector  \
0                                  0   
1                                  0   
2                                  0   
3                                  1   
4                                  0   

   Employment Type_Private Sector/Self Employed  GraduateOrNot_No  \
0                                             1                 0   
1                                             1                 0   
2                                             1                 0   
3                                           

## 검증데이터 분리

In [41]:
# 검증데이터 분리
from sklearn.model_selection import train_test_split
train = train.drop('Unnamed: 0', axis = 1)
test = test.drop('Unnamed: 0', axis = 1)
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size =0.1, random_state= 1204)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(1341, 12) (149, 12) (1341,) (149,)


## 모델 학습 및 예측

In [62]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1000, max_depth=4 ,random_state=2022)
clf = clf.fit(X_tr, y_tr)
pred = clf.predict_proba(X_val)
print(pred[:, 1])

[0.80001091 0.24291917 0.20547688 0.24833221 0.27741143 0.21112211
 0.23606833 0.17961994 0.92205025 0.85329297 0.14581007 0.29869787
 0.85818716 0.263949   0.26579701 0.91843723 0.24965668 0.91031089
 0.23230363 0.25626608 0.2051374  0.84744144 0.24466346 0.30365845
 0.23014094 0.29249932 0.88874823 0.21403419 0.19555836 0.20243348
 0.23889535 0.20567809 0.89233846 0.91858754 0.27579416 0.24392721
 0.27039762 0.24411427 0.26462411 0.2165319  0.21147759 0.21147752
 0.2150766  0.21713976 0.10824018 0.20954442 0.207353   0.22148849
 0.20946633 0.94223588 0.20957248 0.20887089 0.50787011 0.20549046
 0.45438643 0.20744278 0.21737216 0.24580521 0.2093782  0.22632365
 0.18458596 0.24427386 0.2509559  0.44520203 0.21387132 0.20565741
 0.22286234 0.88713517 0.18461529 0.9130476  0.87558596 0.88826854
 0.20822527 0.21002172 0.20285964 0.40443515 0.47672035 0.24730181
 0.21640218 0.77827142 0.17865142 0.20985297 0.22926185 0.2076092
 0.27803126 0.21625063 0.26530425 0.2399182  0.461228   0.20593

In [63]:
# 평가
from sklearn.metrics import roc_auc_score

score = roc_auc_score(y_val, pred[:, 1])
print(score)

0.8233673469387754


## 예측

In [71]:
# test 예측
pred = clf.predict_proba(test)

submit = pd.DataFrame({
    'index' : test.index,
    'y_pred' : pred[:, 1]
})

In [79]:
# csv 파일1 생성 (예시와 다른 형태)
submit.to_csv('123.csv', index= False)

In [80]:
# csv 파일2 생성


In [81]:
# csv 파일1 확인
temp = pd.read_csv('123.csv')
temp

,index,y_pred
0,0,0.187724
1,1,0.174257
2,2,0.149831
3,3,0.932704
4,4,0.182883
...,...,...
492,492,0.252332
493,493,0.915646
494,494,0.243556
495,495,0.935334


In [82]:
# csv 파일2 확인


## 체점
- 수험자는 알 수 없는 부분임

In [69]:
y_test = pd.read_csv("y_test.csv")
roc_auc_score(y_test, pred[:, 1])

0.7875346500346501

In [39]:
# 당신의 score는?: 0.7875693000693